In [1]:
from tqdm import  tqdm
# import wandb
import torch
# from torchsummary import summary
import torchaudio
import torchaudio.transforms as T

from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,


)
from pyctcdecode import build_ctcdecoder

from copy import deepcopy
# import jiwer
from inference import Inference
from datasets  import load_metric
import IPython.display as ipd
from MicRecorder import MicRecorder
from multiprocessing import get_context
torchaudio.set_audio_backend('soundfile')

c:\Users\quang\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cpu


C:\Users\quang\AppData\Local\Temp\ipykernel_8144\2158157440.py:23: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend('soundfile')


In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cpu


In [3]:
model_name = 'checkpoint/checkpoint_Teacher'
processor = Wav2Vec2Processor.from_pretrained('nguyenvulebinh/wav2vec2-base-vietnamese-250h')

In [4]:
def createModel(model_name):
    global inference
    model = Wav2Vec2ForCTC.from_pretrained(model_name)
    model.to(device)
    inference=Inference(model,processor)

In [5]:
createModel(model_name)

Some weights of the model checkpoint at checkpoint/checkpoint_Teacher were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at checkpoint/checkpoint_Teacher and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this mo

In [6]:
# mr=MicRecorder()
# mr.get_recording(5)

In [6]:
from flask import Flask, render_template,request,jsonify
from werkzeug.utils import secure_filename
app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False 

@app.route('/')
def home():
    return render_template('index.html')
@app.route('/recorder', methods=['POST'])
def micRecorder():
    audio_file = request.files['audio']
    filename = secure_filename(audio_file.filename)
    audio_file.save( filename)
    wave, sr = torchaudio.load('recording.wav')
    predict=inference.predict(wave,sr,200)
    return predict
@app.route('/file', methods=['POST'])
def fileRecorder():
    audio_file = request.files['file']
    filename = secure_filename(audio_file.filename)
    audio_file.save( filename)
    wave, sr = torchaudio.load('recording.wav')
    predict=inference.predict(wave,sr,200)
    return predict
@app.route('/model',methods=['POST'])
def updatamodel():
    try:
        model_name=request.get_json().get('data','')
        createModel('checkpoint/'+model_name)
        return jsonify({'message': 'Load model thành công',"status":'1'})
    except :
        return  jsonify({'message': 'Không load được model','status':'0'})
if __name__ == '__main__':
    app.run( port=5007)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5007
Press CTRL+C to quit
127.0.0.1 - - [07/Jan/2024 01:21:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2024 01:21:47] "GET /static/index.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2024 01:21:47] "GET /static/recorder.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2024 01:21:48] "GET /static/Iconka-Saint-Whiskers-Cat-cupid-love.ico HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2024 01:22:01] "GET /static/Iconka-Saint-Whiskers-Cat-cupid-love.ico HTTP/1.1" 304 -
c:\Users\quang\Desktop\AllDemo_dev\NLP\Wav2vec2NST_ASR\inference.py:62: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  pred= self.model(torch.tensor([specs]).to(device))
127.0.0.1 - - [07/Jan/2024 01:22:04] "POST /recorder HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2024 01:22:12] "POST /recorder HTTP/1.1" 